<a href="https://colab.research.google.com/github/muktajoya/InferencePipeline_Huggingface/blob/main/Inferencepipeline_for_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install transformers
!pip install datasets
!pip3 install memory_profiler
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [2]:
import torch

if torch.cuda.is_available():
  device = 0
  torch_device = torch.device('cuda')
  print(device)
else:
  device = -1
  torch_device = torch.device('cpu')
  print(device)

-1


In [3]:
!pip install Starlette
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 1.6 MB/s eta 0:00:00


In [22]:
#dataset from huggingface Hub
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("amandakonet/climatebert-fact-checking")
tokenizer = AutoTokenizer.from_pretrained("amandakonet/climatebert-fact-checking")

In [5]:
from datasets import load_dataset
loader = load_dataset("amandakonet/climate_fever_adopted", split='test').to_pandas()
input_claims = loader['claim'].values.tolist()
input_evidences = loader['evidence'].values.tolist()


Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4298 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/1842 [00:00<?, ? examples/s]

In [24]:
from transformers import pipeline
def predict_by_pipelines(claims: [str], evidences: [str]) -> ([str], [float]):
    def claim_evidence_pairdata():
        for claim, evidence in zip(claims, evidences):
            yield {"text": claim, "text_pair": evidence}

    pipe = pipeline("text-classification",
                    model=model,
                    tokenizer=tokenizer,
                    truncation=True,
                    padding=True,
                    device=-1)
    labels, probs = [], []
    for out in pipe(claim_evidence_pairdata(), batch_size=1):
        print(out)
        labels.append(out['label'])
        probs.append(out['score'])
    return labels, probs

In [23]:
!pip install xformers
pred_labels, pred_probs = predict_by_pipelines(input_claims, input_evidences)


{'label': 'LABEL_1', 'score': 0.6666408777236938}
{'label': 'LABEL_2', 'score': 0.8299382328987122}
{'label': 'LABEL_2', 'score': 0.967482328414917}
{'label': 'LABEL_2', 'score': 0.6380356550216675}
{'label': 'LABEL_2', 'score': 0.7581417560577393}
{'label': 'LABEL_0', 'score': 0.8733140230178833}
{'label': 'LABEL_2', 'score': 0.9167512655258179}
{'label': 'LABEL_0', 'score': 0.8745874166488647}
{'label': 'LABEL_2', 'score': 0.9223232269287109}
{'label': 'LABEL_0', 'score': 0.34526145458221436}
{'label': 'LABEL_2', 'score': 0.3510555922985077}
{'label': 'LABEL_0', 'score': 0.8718220591545105}
{'label': 'LABEL_2', 'score': 0.8575337529182434}
{'label': 'LABEL_2', 'score': 0.8940218687057495}
{'label': 'LABEL_0', 'score': 0.7034542560577393}
{'label': 'LABEL_2', 'score': 0.5990781784057617}
{'label': 'LABEL_2', 'score': 0.9011154174804688}
{'label': 'LABEL_2', 'score': 0.6338019967079163}
{'label': 'LABEL_2', 'score': 0.7875930070877075}
{'label': 'LABEL_2', 'score': 0.9636906385421753}
